In [1]:
import mindspore
from mindspore import Tensor
import mindspore.ops as ops
import mindspore.nn as nn
from mindspore.train.model import Model
from mindspore.nn.metrics import Accuracy
from mindspore.train.callback import LossMonitor, TimeMonitor
import mindspore.dataset as ds
from mindspore import context
from mindspore import dtype as mstype

import numpy as np
import h5py
import json
import re
import moxing as mox
import pickle

INFO:root:Using MoXing-v1.17.3-d858ff4a
INFO:root:Using OBS-Python-SDK-3.20.9.1


In [2]:
context.set_context(mode=context.GRAPH_MODE, device_target='Ascend', device_id=0)

In [3]:
mox.file.copy_parallel(src_url="s3://myobsbrower/nlp/nlp2021-1/annotations", dst_url='./annotations')
mox.file.copy_parallel(src_url="s3://myobsbrower/nlp/nlp2021-1/questions", dst_url='./questions')
mox.file.copy_parallel(src_url="s3://myobsbrower/nlp/nlp2021-1/vocab", dst_url='./vocab')
mox.file.copy_parallel(src_url="s3://myobsbrower/nlp/nlp2021-1/image_features", dst_url='./image_features')
mox.file.copy_parallel(src_url="s3://myobsbrower/nlp/nlp2021-1/dataset.pkl", dst_url='./dataset.pkl')

In [4]:
# class text_encoder(nn.Cell):
#     def __init__(self, vocab_size, embedding_size=2048, max_len=100):
#         super(text_encoder, self).__init__()

#         self.vocab_size = vocab_size
#         self.embedding_size = embedding_size
#         self.max_len = max_len

#         self.embed = nn.Embedding(vocab_size, embedding_size)
#         self.WQ = nn.Dense(embedding_size, embedding_size)
#         self.WK = nn.Dense(embedding_size, embedding_size)
#         self.WV = nn.Dense(embedding_size, embedding_size)

#         self.pe = self.pe_gen()
        
#         self.matmul = ops.BatchMatMul()
#         self.transpose = ops.Transpose()
#         self.softmax = nn.Softmax()
#         self.reducemax = ops.ReduceMax(keep_dims=True)
#         self.print = ops.Print()

#     def pe_gen(self):
#         pe = np.empty((self.max_len, self.embedding_size))
#         even = 10000**(np.arange(self.embedding_size, step=2)/self.embedding_size)
#         odd = 10000**(np.arange(self.embedding_size-1, step=2)/self.embedding_size)
#         for pos in range(self.max_len):
#             pe[pos, ::2] = np.sin(pos/even)
#             pe[pos, 1::2] = np.cos(pos/odd)
#         return Tensor(pe.astype('float32'))

#     def construct(self, x, squeeze=False): # len
#         x = self.embed(x) # batch * len * emb
#         x = x + self.pe[:x.shape[1]] # batch * len * emb
        
#         Q = self.WQ(x) # batch * len * emb
#         K = self.WK(x) # batch * len * emb
#         V = self.WV(x) # batch * len * emb
        
#         QK = self.matmul(Q, self.transpose(K, (0,2,1))) # batch * len * len
#         Z = self.matmul(self.softmax(QK), V) # batch * len * emb
        
#         if squeeze:
#             Z = self.reducemax(Z, 1) # batch * 1 * emb
#         return Z # batch * 1 * emb
class text_encoder(nn.Cell):
    def __init__(self, vocab_size, embedding_size=500, max_len=100):
        super(text_encoder, self).__init__()

        self.vocab_size = vocab_size
        self.embedding_size = embedding_size
        self.max_len = max_len

        self.embed = nn.Embedding(vocab_size, embedding_size)
        self.lstm = nn.LSTM(embedding_size, 2048, 1, has_bias=True, batch_first=True, bidirectional=False)
        self.h0 = Tensor(np.random.randn(1, 128, 2048).astype(np.float32))
        self.c0 = Tensor(np.random.randn(1, 128, 2048).astype(np.float32))
        self.transpose = ops.Transpose()

    def construct(self, x, squeeze=False): # len
        x = self.embed(x) # batch * len * emb
        batch_size = x.shape[0]
        
        _, (hn, cn) = self.lstm(x, (self.h0[:,:batch_size,:], self.c0[:,:batch_size,:]))
        hn = self.transpose(hn, (1,0,2))
        
        return hn

# class text_encoder(nn.Cell):
#     def __init__(self, vocab_size, embedding_size=2048, max_len=100):
#         super(text_encoder, self).__init__()

#         self.vocab_size = vocab_size
#         self.embedding_size = embedding_size
#         self.max_len = max_len

#         self.embed = nn.Embedding(vocab_size, embedding_size)
#         self.WQ = nn.Dense(embedding_size, embedding_size)
#         self.WK = nn.Dense(embedding_size, embedding_size)
#         self.WV = nn.Dense(embedding_size, embedding_size)

#         self.pe = self.pe_gen()
        
#         self.matmul = ops.BatchMatMul()
#         self.transpose = ops.Transpose()
#         self.softmax = nn.Softmax()
#         self.reducemax = ops.ReduceMean(keep_dims=True)
#         self.layernorm = nn.LayerNorm((embedding_size,))
#         self.lstm = nn.LSTM(embedding_size, embedding_size)
#         self.print = ops.Print()
    
#     def pe_gen(self):
#         pe = np.empty((self.max_len, self.embedding_size))
#         even = 10000**(np.arange(self.embedding_size, step=2)/self.embedding_size)
#         odd = 10000**(np.arange(self.embedding_size-1, step=2)/self.embedding_size)
#         for pos in range(self.max_len):
#             pe[pos, ::2] = np.sin(pos/even)
#             pe[pos, 1::2] = np.cos(pos/odd)
#         return Tensor(pe.astype('float32'))
    
#     def construct(self, x, squeeze=False): # len
#         x = self.embed(x) # batch * len * emb
#         batch_size = x.shape[0]
#         h = Tensor(np.zeros([1, batch_size, self.embedding_size]).astype(np.float32))
#         c = Tensor(np.zeros([1, batch_size, self.embedding_size]).astype(np.float32))
#         Z = x
#         if squeeze:
#             _, (Z, __) = self.lstm(self.transpose(Z, (1,0,2)), (self.h[:, :x.shape[0]], self.c[:, :x.shape[0]])) # 1 * batch * emb
#             Z = self.transpose(Z, (1,0,2)) # batch * 1 * emb
#             # Z = self.reducemax(Z, 1) # batch * 1 * emb
#         return Z # batch * 1 * emb

In [5]:
class block(nn.Cell):
    def __init__(self, input_size=2048, hidden_size=512):
        super(block, self).__init__()
        self.Wq = nn.Dense(input_size, hidden_size)
        self.Wi = nn.Dense(input_size, hidden_size, has_bias=False)
        self.Wp = nn.Dense(hidden_size, 1)
        self.transpose = ops.Transpose()
        self.tanh = nn.Tanh()
        self.softmax = nn.Softmax(0)
        self.reduce_sum = ops.ReduceSum()
        self.matmul = ops.BatchMatMul()
        self.dropout = nn.Dropout()
    
    def construct(self, v_i, v_q):
        encoded_q = self.Wq(v_q) # batch * 1 * hid
        encoded_i = self.Wi(self.transpose(v_i, (0,2,1))) # batch * 36 * hid
        hA = self.tanh(encoded_q + encoded_i) # batch * 36 * hid
        hA = self.dropout(hA)
        pI = self.softmax(self.Wp(hA)) # batch * 36 * 1
        #vI = self.matmul(self.transpose(pI, (0,2,1)), hA) # batch * 1 * hid
        vI = self.matmul(self.transpose(pI, (0,2,1)), self.transpose(v_i, (0,2,1)))
        u = vI + v_q
        return u

class SAN(nn.Cell):
    def __init__(self, hidden_size=2048):
        super(SAN, self).__init__()
        
        self.block0 = block(hidden_size, 512)
        self.block1 = block(hidden_size, 512)
        #self.block2 = block(hidden_size)
        #self.block3 = block(hidden_size)

    def construct(self, v_i, v_q): # v_i: batch * 36 * hid, v_q: batch * 1 * hid
        u0 = self.block0(v_i, v_q)
        u1 = self.block1(v_i, u0)
        #u2 = self.block2(v_i, u1)
        #u3 = self.block3(v_i, u2)

        return u1 # 1 * hid

In [6]:
class VQA(nn.Cell):
    def __init__(self, question_vocab_size, answer_vocab_size, nb_answers, hidden_size=2048):
        super(VQA, self).__init__()
        self.question_encoder = text_encoder(question_vocab_size, 500)
        self.answer_encoder = text_encoder(answer_vocab_size, 500)
        self.SAN = SAN(hidden_size)
        self.classifier = nn.Dense(hidden_size, nb_answers)
        
        self.matmul = ops.BatchMatMul()
        self.transpose = ops.Transpose()
        self.argmax = ops.Argmax()
        self.softmax = nn.Softmax()
        self.cast = ops.Cast()
        self.print = ops.Print()
        self.dropout = nn.Dropout()

    def construct(self, x):
        v_i = x[:, :-1]
        q = self.cast(x[:, -2:-1, :], mindspore.int32).view(x.shape[0],-1) # batch * 1 * 36
        v_q = self.question_encoder(q, True) # batch * 1 * hid
        uk = self.SAN(v_i, v_q) # batch * 1 * hid
        score = self.classifier(uk) # batch * 1 * n
        score = self.dropout(score)
        prob = self.softmax(score) # batch * 1 * n
        chosen = self.argmax(score)
        
        return score.view(x.shape[0], -1)
        # return score
        # return prob.view(x.shape[0], -1)
        # return prob
        # return chosen

In [7]:
categories = ['train', 'val', 'test']

In [8]:
questions = {}
annotations = {}
features = {}

for c in categories:
    quest_json = open('./questions/{}.json'.format(c))
    quest = json.load(quest_json)
    quest_json.close()
    questions[c] = quest['questions']

    anno_json = open('./annotations/{}.json'.format(c))
    anno = json.load(anno_json)
    anno_json.close()
    annotations[c] = anno['annotations']

    try:
        features[c] = h5py.File('./image_features/image-{}.h5'.format(c), 'r')
    except:
        pass

vocab_file = open('./vocab/test.json')
vocab = json.load(vocab_file)
vocab_file.close()

In [9]:
_special_chars = re.compile('[^a-z0-9 ]*')
_period_strip = re.compile(r'(?!<=\d)(\.)(?!\d)')
_comma_strip = re.compile(r'(\d)(,)(\d)')
_punctuation_chars = re.escape(r';/[]"{}()=+\_-><@`,?!')
_punctuation = re.compile(r'([{}])'.format(re.escape(_punctuation_chars)))
_punctuation_with_a_space = re.compile(r'(?<= )([{0}])|([{0}])(?= )'.format(_punctuation_chars))

def question_tokenize(question):
    question = question.lower()[:-1]
    question = _special_chars.sub('', question)
    return question.split(' ')

def answer_tokenize(answer):
    if _punctuation.search(answer) is None:
        return answer
    answer = _punctuation_with_a_space.sub('', answer)
    if re.search(_comma_strip, answer) is not None:
        answer = answer.replace(',', '')
    answer = _punctuation.sub(' ', answer)
    answer = _period_strip.sub('', answer)
    return answer.strip()

def question2vec(question):
    return Tensor([vocab['question'][word] for word in question_tokenize(question)])

def answer2vec(answer):
    return Tensor([vocab['answer'][answer_tokenize(answer)]])

In [10]:
data_file = open('dataset.pkl', 'rb')
dataset = pickle.load(data_file)
data_file.close()

In [11]:
nb_answers = max([max([data['answer_label'] for data in dataset[c]]) for c in categories]) + 1

In [12]:
nb_choices = 16
nb_epoches = 10

In [13]:
vqa = VQA(len(vocab['question']), len(vocab['answer']), nb_answers)

In [14]:
id2idx = {'train' : {}, 'test' : {}, 'val' : {}}
for c in categories:
    id2idx[c] = {id : i for i, id in enumerate(features[c]['ids'])}

In [15]:
id2idx_file = open('id2idx.pkl', 'wb')
pickle.dump(id2idx, id2idx_file)
id2idx_file.close()
mox.file.copy_parallel(src_url="./id2idx.pkl", dst_url='s3://myobsbrower/nlp/nlp2021-1/id2idx.pkl')

In [16]:
stop_index = 8717

In [17]:
input_data = {'train' : [], 'test' : [], 'val' : []}
for c in categories:
    for i, data in enumerate(dataset[c]):
        if i >= 10000:
            break
        question = np.ones((36,), 'int32') * stop_index
        question_vec = data['question'].astype('int32')
        question[:question_vec.shape[0]] = question_vec
        image_text = np.concatenate((features[c]['features'][id2idx[c][data['image_id']]].astype('float32'), 
                                    question.astype('float32').reshape(1,-1)), 0)
        input_data[c].append(
            (image_text, np.array([data['answer_label']], 'int32')[0])
        )

In [18]:
class Generator():
    def __init__(self, input_list):
        self.input_list=input_list
    def __getitem__(self,item):
        return (self.input_list[item][0], self.input_list[item][1])
    def __len__(self):
        return len(self.input_list)

In [19]:
input_ds = {}
for c in categories:
    input_ds[c] = ds.GeneratorDataset(
        source=Generator(input_list=input_data[c]), 
        column_names=["data", "label"],
        shuffle=False
    )
    input_ds[c] = input_ds[c].batch(batch_size=64)
    # if c == 'train':
        # input_ds[c] = input_ds[c].repeat(nb_epoches)

In [20]:
opt = nn.Adam(vqa.get_parameters(), learning_rate=0.001, weight_decay=3e-5)
# opt = nn.Momentum(vqa.get_parameters(), 0.001, 0.9)
loss = nn.SoftmaxCrossEntropyWithLogits(sparse=True)
model = Model(vqa, loss_fn=loss, optimizer=opt, metrics={'acc': Accuracy()})

In [21]:
time_cb = TimeMonitor(data_size=64)
loss_cb = LossMonitor()

In [ ]:
model.train(20, input_ds['train'], callbacks=[time_cb, loss_cb])
print('train_success')

epoch: 1 step: 157, loss is 7.4612937
epoch time: 109855.398 ms, per step time: 699.716 ms
epoch: 2 step: 157, loss is 5.8511934
epoch time: 70623.371 ms, per step time: 449.830 ms
epoch: 3 step: 157, loss is 5.642644
epoch time: 70627.251 ms, per step time: 449.855 ms
epoch: 4 step: 157, loss is 5.104364
epoch time: 70624.819 ms, per step time: 449.840 ms
epoch: 5 step: 157, loss is 6.354743
epoch time: 70627.364 ms, per step time: 449.856 ms
epoch: 6 step: 157, loss is 4.7461033
epoch time: 70629.914 ms, per step time: 449.872 ms
epoch: 7 step: 157, loss is 4.9699254
epoch time: 70629.806 ms, per step time: 449.871 ms
epoch: 8 step: 157, loss is 5.4195204
epoch time: 70626.843 ms, per step time: 449.853 ms
epoch: 9 step: 157, loss is 3.763872
epoch time: 70646.902 ms, per step time: 449.980 ms
epoch: 10 step: 157, loss is 4.8594546
epoch time: 70638.940 ms, per step time: 449.930 ms
epoch: 11 step: 157, loss is 4.642271
epoch time: 70628.031 ms, per step time: 449.860 ms
epoch: 12 st

In [23]:
acc = model.eval(input_ds['test'])
print("accuracy: ", acc)

accuracy:  {'acc': 0.12470143312101911}


In [22]:
from mindspore.train.callback import ModelCheckpoint
ckpt_cb = ModelCheckpoint()
model.train(1, input_ds['train'], callbacks=[time_cb, loss_cb, ckpt_cb],dataset_sink_mode=False)
print('train_success')

epoch: 1 step: 1, loss is 8.672701
epoch time: 51597.254 ms, per step time: 51597.254 ms
train_success


In [24]:
mox.file.copy(src_url="./CKP-1_694.ckpt", dst_url='s3://myobsbrower/nlp/nlp2021-1/CKP-1_694.ckpt')

In [23]:
acc = model.eval(input_ds['test'], dataset_sink_mode=False)
print("accuracy: ", acc)

accuracy:  {'acc': 0.1875}


In [24]:
model.train(1, input_ds['train'], callbacks=[time_cb, loss_cb, ckpt_cb], dataset_sink_mode=False)
print('the second train succeeds')

epoch: 1 step: 1, loss is 6.4220457
epoch time: 2738.978 ms, per step time: 2738.978 ms
the second train succeeds


In [25]:
acc = model.eval(input_ds['test'], dataset_sink_mode=False)
print("accuracy: ", acc)

accuracy:  {'acc': 0.1875}


In [26]:
!ls

annotations   CKP-graph.meta  id2idx.pkl      kernel_meta  somas_meta
CKP-1_1.ckpt  dataset.pkl     image_features  questions    vocab


In [27]:
from mindspore import load_checkpoint, load_param_into_net

param_dict = load_checkpoint("CKP-1_1.ckpt")
load_param_into_net(vqa, param_dict)

acc = model.eval(input_ds['test'], dataset_sink_mode=False)

In [28]:
print("accuracy: ", acc)

accuracy:  {'acc': 0.1875}


In [29]:
from mindspore import load_checkpoint, load_param_into_net

param_dict = load_checkpoint("CKP-1_1.ckpt")
load_param_into_net(vqa, param_dict)

acc = model.eval(input_ds['test'], dataset_sink_mode=False)
print("accuracy: ", acc)

accuracy:  {'acc': 0.1875}


In [30]:
model.train(1, input_ds['train'], callbacks=[time_cb, loss_cb, ckpt_cb], dataset_sink_mode=False)
print('the third train succeeds')

epoch: 1 step: 1, loss is 6.4220457
epoch time: 2736.719 ms, per step time: 2736.719 ms
the third train succeeds


In [25]:
mox.file.copy(src_url="s3://myobsbrower/nlp/nlp2021-1/CKP-1_694.ckpt", dst_url='./CKP-1_694.ckpt')

In [33]:
from mindspore import load_checkpoint, load_param_into_net

vqa1 = VQA(len(vocab['question']), len(vocab['answer']), nb_answers)
param_dict = load_checkpoint("CKP-1_694.ckpt")
load_param_into_net(vqa1, param_dict)

opt1 = nn.Adam(vqa1.get_parameters(), learning_rate=0.0001)
loss1 = nn.SoftmaxCrossEntropyWithLogits(sparse=True)
model1 = Model(vqa1, loss_fn=loss1, optimizer=opt1, metrics={'acc': Accuracy()})

acc = model1.eval(input_ds['test'], dataset_sink_mode=False)
print("accuracy: ", acc)

accuracy:  {'acc': 0.148}
